In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import sys
import os
print(os.listdir("../input"))
import gc
# Any results you write to the current directory are saved as output.

In [2]:
dtypes = {'ip': 'uint32','app': 'uint16','device': 'uint16','os': 'uint16','channel': 'uint16','is_attributed' : 'uint8'}
df1 = pd.read_csv("../input/test_supplement.csv", dtype=dtypes, usecols=['ip','app','device','os', 'channel', 'click_time'])
df2 = pd.read_csv("../input/test.csv", dtype=dtypes, usecols=['ip','app','device','os', 'channel', 'click_time'])


In [3]:
df1['click_time']= pd.to_datetime(df1['click_time'])

In [4]:
df1=df1[df1['click_time'].dt.day==10]
df1.shape

In [21]:
df1[(df1['click_time'].dt.hour>=4)&(df1['click_time'].dt.hour<=15)].shape

In [1]:
df1.drop_duplicates().shape

In [23]:
df1.tail()

In [7]:
df2.head()

In [18]:
df2.tail()

In [22]:
print('this is {}'.format((1,2)))

In [ ]:
most_freq_hours_in_test_data = [4,5,9,10,13,14]
least_freq_hours_in_test_data = [6, 11, 15]

def add_counts(df, cols):
    arr_slice = df[cols].values
    unq, unqtags, counts = np.unique(np.ravel_multi_index(arr_slice.T, arr_slice.max(axis=0)+1), return_inverse=True, return_counts=True)
    df["_".join(cols)+"_count"] = counts[unqtags]

def add_next_click(df):
    D = 2**26
    df['category'] = (df['ip'].astype(str) + "_" + df['app'].astype(str) + "_" + df['device'].astype(str) + "_" + df['os'].astype(str)).apply(hash) % D
    click_buffer = np.full(D, 3000000000, dtype=np.uint32)
    df['epochtime'] = df['click_time'].astype(np.int64) // 10 ** 9
    next_clicks = []
    for category, time in zip(reversed(df['category'].values), reversed(df['epochtime'].values)):
        next_clicks.append(click_buffer[category] - time)
        click_buffer[category] = time
    del click_buffer
    df['next_click'] = list(reversed(next_clicks))
    df.drop(['category', 'epochtime'], axis=1, inplace=True)

def ip_analysis(df):
    
    #Extrace date info
    df['click_time']= pd.to_datetime(df['click_time'])
    df['hour'] = df['click_time'].dt.hour.astype('uint8')
    df['day'] = df['click_time'].dt.day.astype('uint8')
    df['wday'] = df['click_time'].dt.dayofweek.astype('uint8')
    gc.collect()
    
    df['in_test_hh'] = ( 3 - 2 * df['hour'].isin( most_freq_hours_in_test_data ) - 1 * df['hour'].isin( least_freq_hours_in_test_data )).astype('uint8')

    print('Grouping...')
    
    add_counts(df, ['ip'])
    add_counts(df, ['os', 'device'])
    add_counts(df, ['os', 'app', 'channel'])
    add_counts(df, ['ip', 'device'])
    add_counts(df, ['app', 'channel'])
    add_counts(df, ['ip', 'wday', 'in_test_hh'])
    add_counts(df, ['ip', 'wday', 'hour'])
    add_counts(df, ['ip', 'os', 'wday', 'hour'])
    add_counts(df, ['ip', 'app', 'wday', 'hour'])
    add_counts(df, ['ip', 'device', 'wday', 'hour'])
    add_counts(df, ['ip', 'app', 'os'])
    add_counts(df, ['wday', 'hour', 'app'])
    
    #Groups
    print('Adding next_click...')
    add_next_click(df)
    
    df.drop(['ip', 'day', 'click_time'], axis=1, inplace=True )
    gc.collect()

    #print( df.info() )

    return df

In [ ]:
def extra_test():
  print('Loading the test and test_supplement data...')

  df1 = pd.read_csv('../input/test_supplement.csv', dtype=dtypes, usecols=['ip','app','device','os', 'channel', 'click_time'])
  df2 = pd.read_csv("../input/test.csv", dtype=dtypes, usecols=['ip','app','device','os', 'channel', 'click_time'])
  df1=df1.sample(17000000)
  gc.collect()
  merge_features = list(df1.columns)
  df = pd.merge(df1, df2, on=merge_features, how='outer', indicator=True)
  df['click_id'] = 0
  df2['click_id'] = 1
  
  return df[df['_merge']=='left_only'].drop('_merge',axis=1).append(df2)
     